In [1]:
### LIBRARIES ###
import os
import mne
import pandas as pd
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report, get_entities_from_fname
import scipy.signal as signal
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import bids
from mne.datasets.eegbci import standardize
from fooof.utils import trim_spectrum
#For plots outside jupiter
get_ipython().run_line_magic("matplotlib", " qt ")

c:\Users\acer\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\acer\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\acer\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


#### DF

##### Medellin

In [8]:
antioquia={
'layout':{'extension':'.fif', 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'O1', 'OZ', 'O2'],
    'path':'D:/EEGs/PhD_datasets/BIDS/antioquia/'
}

#Path of the BIDS folder
bids_root = 'D:/EEGs/PhD_datasets/BIDS/antioquia/'
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=antioquia

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]
print(len(eegs), eegs)

72 ['D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-001\\eeg\\sub-001_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-002\\eeg\\sub-002_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-003\\eeg\\sub-003_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-004\\eeg\\sub-004_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-005\\eeg\\sub-005_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-006\\eeg\\sub-006_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-007\\eeg\\sub-007_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-008\\eeg\\sub-008_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datas

Averaged epochs approach

In [80]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(44)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        features = {}
        srate = downsample
        n_fft, n_overlap, n_per_seg= int(2*srate), int(srate), int(2*srate)
        psd, freqs = mne.time_frequency.psd_array_welch(epochs.get_data()[:,ch,:], sfreq = srate, n_per_seg=n_per_seg, fmin=1, fmax=30, n_overlap = n_overlap, n_fft= n_fft, average='mean', verbose = False)
        psds_mean = psd.mean(0)
        psd = psds_mean
        al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
        icf_ind = np.argmax(np.abs(al_psd))
        al_icf = al_freqs[icf_ind]
        dominant_freq = al_icf
        features['center'] = 'medellin'
        features['subject'] = ('m_' + subject)
        features['channel'] = ch_label
        features['df_ep'] = dominant_freq
        dict_list.append(features)
dominant_average = pd.DataFrame(dict_list)

Wall time: 2min 41s


FDA Approach

In [81]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(44)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            srate = downsample #This gets the sampling rate of the EEGLAB object
            nfft, noverlap, nperseg = int(2*srate), int(srate), int(2*srate)
            freqs, psd = signal.welch(epochs._data[ep,ch,:], fs=srate,  nfft=nfft, noverlap=noverlap, nperseg=nperseg, window='hamming', scaling='spectrum', average='mean') #This calculates PSD Welch
            al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
            icf_ind = np.argmax(np.abs(al_psd))
            al_icf = al_freqs[icf_ind]
            dominant_freq = al_icf
            features = {}
            features['center'] = 'medellin'
            features['subject'] = ('m_' + subject)
            features['channel'] = ch_label
            features['epoch'] = ep
            features['df_ep'] = dominant_freq
            dict_list.append(features)
            df_epochs = pd.DataFrame(dict_list)
            df_epochs['dfv_ch'] = abs(df_epochs.df_ep - df_epochs.groupby(['subject','channel'])['df_ep'].shift(periods=1))
df_epochs['dfv_ch'] = df_epochs['dfv_ch'].fillna(0)

dfv_average = pd.DataFrame(df_epochs.groupby(['subject', 'channel']).df_ep.std())
dfv_average.rename(columns = {'df_ep':'dfv_ave'}, inplace = True)
df_dfv_ave = pd.merge(dominant_average, dfv_average, on=['subject', 'channel'])

df_dfv_ave.to_csv('D:/Papers/FDA_spectral/review/reanalysis/medellin/df_averaged.csv', index=False)
df_epochs.to_csv('D:/Papers/FDA_spectral/review/reanalysis/medellin/df_fda.csv', index=False)

Wall time: 5h 27min 28s


##### California #####

In [2]:
california={
'layout':{'extension':'.fif', 'session':['hc', 'on'], 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['FP1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5', 'P7', 'P3', 'PZ', 'PO3', 'O1', 'OZ', 'O2', 'PO4', 'P4', 'P8', 'CP6', 'CP2', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'AF4', 'FP2', 'FZ', 'CZ'],
    'path':'D:/EEGs/PhD_datasets/BIDS/california/'
}

#Path of the BIDS folder
bids_root = 'D:/EEGs/PhD_datasets/BIDS/california/'
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=california #DEFINE DATASET

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]

print(len(eegs), eegs)

31 ['D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc1\\ses-hc\\eeg\\sub-hc1_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc2\\ses-hc\\eeg\\sub-hc2_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc4\\ses-hc\\eeg\\sub-hc4_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc7\\ses-hc\\eeg\\sub-hc7_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc8\\ses-hc\\eeg\\sub-hc8_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc10\\ses-hc\\eeg\\sub-hc10_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc18\\ses-hc\\eeg\\sub-hc18_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\B

Averaged epochs approach

In [56]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(28)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        features = {}
        srate = downsample
        n_fft, n_overlap, n_per_seg= int(2*srate), int(srate), int(2*srate)
        psd, freqs = mne.time_frequency.psd_array_welch(epochs.get_data()[:,ch,:], sfreq = srate, n_per_seg=n_per_seg, fmin=1, fmax=30, n_overlap = n_overlap, n_fft= n_fft, average='mean', verbose = False)
        psds_mean = psd.mean(0)
        psd = psds_mean
        al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
        icf_ind = np.argmax(np.abs(al_psd))
        al_icf = al_freqs[icf_ind]
        dominant_freq = al_icf
        features['center'] = 'california'
        features['subject'] = ('c_' + subject)
        features['channel'] = ch_label
        features['df_ep'] = dominant_freq
        dict_list.append(features)
dominant_average = pd.DataFrame(dict_list)

Wall time: 17.3 s


FDA Approach

In [58]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(28)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            srate = downsample #This gets the sampling rate of the EEGLAB object
            nfft, noverlap, nperseg = int(2*srate), int(srate), int(2*srate)
            freqs, psd = signal.welch(epochs._data[ep,ch,:], fs=srate,  nfft=nfft, noverlap=noverlap, nperseg=nperseg, window='hamming', scaling='spectrum', average='mean') #This calculates PSD Welch
            al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
            icf_ind = np.argmax(np.abs(al_psd))
            al_icf = al_freqs[icf_ind]
            dominant_freq = al_icf
            features = {}
            features['center'] = 'california'
            features['subject'] = ('c_' + subject)
            features['channel'] = ch_label
            features['epoch'] = ep
            features['df_ep'] = dominant_freq
            dict_list.append(features)
            df_epochs = pd.DataFrame(dict_list)
            df_epochs['dfv_ch'] = abs(df_epochs.df_ep - df_epochs.groupby(['subject','channel'])['df_ep'].shift(periods=1))
df_epochs['dfv_ch'] = df_epochs['dfv_ch'].fillna(0)

dfv_average = pd.DataFrame(df_epochs.groupby(['subject', 'channel']).df_ep.std())
dfv_average.rename(columns = {'df_ep':'dfv_ave'}, inplace = True)
df_dfv_ave = pd.merge(dominant_average, dfv_average, on=['subject', 'channel'])

df_dfv_ave.to_csv('D:/Papers/FDA_spectral/review/reanalysis/california/df_averaged.csv', index=False)
df_epochs.to_csv('D:/Papers/FDA_spectral/review/reanalysis/california/df_fda.csv', index=False)

Wall time: 9min 5s


##### Iowa

In [ ]:
iowa={
'layout':{'extension':'.fif', 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['T8', 'CP3', 'T7', 'C4', 'C5', 'FT8', 'F6', 'FC5', 'FC3', 'C2', 'TP9', 'F7', 'C6', 'CP6', 'F2', 'TP7', 'PO7', 'FCZ', 'O2', 'P2', 'CZ', 'F1', 'P6', 'AF8', 'C3', 'AF4', 'CPZ', 'F5', 'CP1', 'O1', 'P8', 'FC6', 'FC2', 'P4', 'F8', 'FP2', 'AF3', 'POZ', 'FT10', 'P1', 'FC1', 'TP10', 'FP1', 'AF7', 'PO8', 'TP8', 'F4', 'FZ', 'AFZ', 'OZ', 'CP4', 'P3', 'FT7', 'CP5', 'P5', 'P7', 'F3', 'FC4', 'CP2', 'C1'],
    'path':'D:/EEGs/PhD_datasets/BIDS/iowa/'
}

#Path of the BIDS folder
bids_root = 'D:/EEGs/PhD_datasets/BIDS/iowa/'
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=iowa #DEFINE DATASET

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]

print(len(eegs), eegs)

Averaged epochs approach

In [72]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(16)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        features = {}
        srate = downsample
        n_fft, n_overlap, n_per_seg= int(2*srate), int(srate), int(2*srate)
        psd, freqs = mne.time_frequency.psd_array_welch(epochs.get_data()[:,ch,:], sfreq = srate, n_per_seg=n_per_seg, fmin=1, fmax=30, n_overlap = n_overlap, n_fft= n_fft, average='mean', verbose = False)
        psds_mean = psd.mean(0)
        psd = psds_mean
        al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
        icf_ind = np.argmax(np.abs(al_psd))
        al_icf = al_freqs[icf_ind]
        dominant_freq = al_icf
        features['center'] = 'iowa'
        features['subject'] = ('i_' + subject)
        features['channel'] = ch_label
        features['df_ep'] = dominant_freq
        dict_list.append(features)
dominant_average = pd.DataFrame(dict_list)

Wall time: 26.9 s


FDA Approach

In [73]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(16)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            srate = downsample #This gets the sampling rate of the EEGLAB object
            nfft, noverlap, nperseg = int(2*srate), int(srate), int(2*srate)
            freqs, psd = signal.welch(epochs._data[ep,ch,:], fs=srate,  nfft=nfft, noverlap=noverlap, nperseg=nperseg, window='hamming', scaling='spectrum', average='mean') #This calculates PSD Welch
            al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
            icf_ind = np.argmax(np.abs(al_psd))
            al_icf = al_freqs[icf_ind]
            dominant_freq = al_icf
            features = {}
            features['center'] = 'iowa'
            features['subject'] = ('i_' + subject)
            features['channel'] = ch_label
            features['epoch'] = ep
            features['df_ep'] = dominant_freq
            dict_list.append(features)
            df_epochs = pd.DataFrame(dict_list)
            df_epochs['dfv_ch'] = abs(df_epochs.df_ep - df_epochs.groupby(['subject','channel'])['df_ep'].shift(periods=1))
df_epochs['dfv_ch'] = df_epochs['dfv_ch'].fillna(0)

dfv_average = pd.DataFrame(df_epochs.groupby(['subject', 'channel']).df_ep.std())
dfv_average.rename(columns = {'df_ep':'dfv_ave'}, inplace = True)
df_dfv_ave = pd.merge(dominant_average, dfv_average, on=['subject', 'channel'])

df_dfv_ave.to_csv('D:/Papers/FDA_spectral/review/reanalysis/iowa/df_averaged.csv', index=False)
df_epochs.to_csv('D:/Papers/FDA_spectral/review/reanalysis/iowa/df_fda.csv', index=False)

Wall time: 9min 31s


##### Finland ####

In [ ]:
finland={
'layout':{'extension':'.fif', 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'FZ', 'CZ', 'PZ', 'IZ', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'FT9', 'FT10', 'TP9', 'TP10', 'F1', 'F2', 'C1', 'C2', 'P1', 'P2', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'AF7', 'AF8', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'FPZ', 'CPZ', 'POZ', 'OZ'],
    'path':'D:/EEGs/PhD_datasets/BIDS/finland/'
}

#Path of the BIDS folder
bids_root = 'D:/EEGs/PhD_datasets/BIDS/finland/'
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=finland #DEFINE DATASET

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]

print(len(eegs), eegs)

Averaged epochs approach

In [77]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(18)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        features = {}
        srate = downsample
        n_fft, n_overlap, n_per_seg= int(2*srate), int(srate), int(2*srate)
        psd, freqs = mne.time_frequency.psd_array_welch(epochs.get_data()[:,ch,:], sfreq = srate, n_per_seg=n_per_seg, fmin=1, fmax=30, n_overlap = n_overlap, n_fft= n_fft, average='mean', verbose = False)
        psds_mean = psd.mean(0)
        psd = psds_mean
        al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
        icf_ind = np.argmax(np.abs(al_psd))
        al_icf = al_freqs[icf_ind]
        dominant_freq = al_icf
        features['center'] = 'finland'
        features['subject'] = ('f_' + subject)
        features['channel'] = ch_label
        features['df_ep'] = dominant_freq
        dict_list.append(features)
dominant_average = pd.DataFrame(dict_list)

Wall time: 29 s


FDA Approach

In [78]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(18)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            srate = downsample #This gets the sampling rate of the EEGLAB object
            nfft, noverlap, nperseg = int(2*srate), int(srate), int(2*srate)
            freqs, psd = signal.welch(epochs._data[ep,ch,:], fs=srate,  nfft=nfft, noverlap=noverlap, nperseg=nperseg, window='hamming', scaling='spectrum', average='mean') #This calculates PSD Welch
            al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
            icf_ind = np.argmax(np.abs(al_psd))
            al_icf = al_freqs[icf_ind]
            dominant_freq = al_icf
            features = {}
            features['center'] = 'finland'
            features['subject'] = ('f_' + subject)
            features['channel'] = ch_label
            features['epoch'] = ep
            features['df_ep'] = dominant_freq
            dict_list.append(features)
            df_epochs = pd.DataFrame(dict_list)
            df_epochs['dfv_ch'] = abs(df_epochs.df_ep - df_epochs.groupby(['subject','channel'])['df_ep'].shift(periods=1))
df_epochs['dfv_ch'] = df_epochs['dfv_ch'].fillna(0)

dfv_average = pd.DataFrame(df_epochs.groupby(['subject', 'channel']).df_ep.std())
dfv_average.rename(columns = {'df_ep':'dfv_ave'}, inplace = True)
df_dfv_ave = pd.merge(dominant_average, dfv_average, on=['subject', 'channel'])

df_dfv_ave.to_csv('D:/Papers/FDA_spectral/review/reanalysis/finland/df_averaged.csv', index=False)
df_epochs.to_csv('D:/Papers/FDA_spectral/review/reanalysis/finland/df_fda.csv', index=False)

Wall time: 21min 15s


#### Bandpower

In [ ]:
def bandpower(data, sf, band, method='multitaper_average', window_sec=None, relative=False):
    """Compute the average power of the signal x in a specific frequency band.

    Requires MNE-Python >= 0.14.

    Parameters
    ----------
    data : 1d-array
      Input signal in the time-domain.
    sf : float
      Sampling frequency of the data.
    band : list
      Lower and upper frequencies of the band of interest.
    method : string
      Periodogram method: 'multitaper' or 'multitaper_average'. Default is 'multitaper_average'
    window_sec : float
      Length of each window in seconds. Useful only if method == 'welch'.
      If None, window_sec = (1 / min(band)) * 2.
    relative : boolean
      If True, return the relative power (= divided by the total power of the signal).
      If False (default), return the absolute power.

    Return
    ------
    bp : float
      Absolute or relative band power.
    """
    from scipy.integrate import simps
    from mne.time_frequency import psd_array_multitaper

    band = np.asarray(band)
    low, high = band

    # Compute PSD using multitaper average psd vectors over all epochs, or return the psd vector for each epoch
    if method == 'multitaper_average':
        psd, freqs = psd_array_multitaper(data, sf, adaptive=True, low_bias = True,
                                          normalization='full', verbose=0)
        psds_mean = psd.mean(0)
        psd = psds_mean

    elif method == 'multitaper':
        psd, freqs = psd_array_multitaper(data, sf, adaptive=True, low_bias = True,
                                          normalization='full', verbose=0)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp

##### Medellin

In [ ]:
antioquia={
'layout':{'extension':'.fif', 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'O1', 'OZ', 'O2'],
    'path':'D:/EEGs/PhD_datasets/BIDS/antioquia/'
}

#Path of the BIDS folder
bids_root = 'D:/EEGs/PhD_datasets/BIDS/antioquia/'
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=antioquia

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]
print(len(eegs), eegs)

Averaged epochs approach

In [ ]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(44)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    sf = downsample
    for ch,ch_label in enumerate(channels):
        features = {}
        features['center'] = 'medellin'
        features['subject'] = ('m_' + subject)
        features['channel'] = ch_label
        features['delta'] = bandpower(epochs.get_data()[:,ch,:], sf, [1, 4], 'multitaper_average', relative=True)
        features['theta'] = bandpower(epochs.get_data()[:,ch,:], sf, [4, 8], 'multitaper_average', relative=True)
        features['alpha'] = bandpower(epochs.get_data()[:,ch,:], sf, [8, 13], 'multitaper_average', relative=True)
        features['beta'] = bandpower(epochs.get_data()[:,ch,:], sf, [13, 30], 'multitaper_average', relative=True)
        features['pre_alpha'] = bandpower(epochs.get_data()[:,ch,:], sf, [5.5, 8], 'multitaper_average', relative=True)
        features['slow_theta'] = bandpower(epochs.get_data()[:,ch,:], sf, [4, 5.5], 'multitaper_average', relative=True)
        features['alpha_theta'] = bandpower(epochs.get_data()[:,ch,:], sf, [8, 13], 'multitaper_average', relative=True) / bandpower(epochs.get_data()[:,ch,:], sf, [4, 8], 'multitaper_average', relative=True)
        dict_list.append(features)
bandpowers = pd.DataFrame(dict_list)

bandpowers.to_csv('D:/Papers/FDA_spectral/review/reanalysis/medellin/bandpowers_averaged.csv', index=False)


Wall time: 2h 50min 57s


FDA Approach

In [ ]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(44)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    sf = downsample
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            features = {}
            features['center'] = 'medellin'
            features['subject'] = ('m_' + subject)
            features['channel'] = ch_label
            features['epoch'] = ep
            features['delta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [1, 4], 'multitaper', relative=True)
            features['theta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [4, 8], 'multitaper', relative=True)
            features['alpha'] = bandpower(epochs.get_data()[ep,ch,:], sf, [8, 13], 'multitaper', relative=True)
            features['beta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [13, 30], 'multitaper', relative=True)
            features['pre_alpha'] = bandpower(epochs.get_data()[ep,ch,:], sf, [5.5, 8], 'multitaper', relative=True)
            features['slow_theta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [4, 5.5], 'multitaper', relative=True)
            features['alpha_theta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [8, 13], 'multitaper', relative=True) / bandpower(epochs.get_data()[ep,ch,:], sf, [4, 8], 'multitaper', relative=True)
            dict_list.append(features)
bandpowers_fda = pd.DataFrame(dict_list)

bandpowers_fda.to_csv('D:/Papers/FDA_spectral/review/reanalysis/medellin/bandpowers_fda.csv', index=False)

Wall time: 7h 43min 52s


##### California #####

In [24]:
california={
'layout':{'extension':'.fif', 'session':['hc', 'on'], 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['FP1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5', 'P7', 'P3', 'PZ', 'PO3', 'O1', 'OZ', 'O2', 'PO4', 'P4', 'P8', 'CP6', 'CP2', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'AF4', 'FP2', 'FZ', 'CZ'],
    'path':'D:/EEGs/PhD_datasets/BIDS/california/'
}

#Path of the BIDS folder
bids_root = 'D:/EEGs/PhD_datasets/BIDS/california/'
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=california #DEFINE DATASET

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]

print(len(eegs), eegs)

31 ['D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc1\\ses-hc\\eeg\\sub-hc1_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc2\\ses-hc\\eeg\\sub-hc2_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc4\\ses-hc\\eeg\\sub-hc4_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc7\\ses-hc\\eeg\\sub-hc7_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc8\\ses-hc\\eeg\\sub-hc8_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc10\\ses-hc\\eeg\\sub-hc10_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc18\\ses-hc\\eeg\\sub-hc18_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\B

Averaged epochs approach

In [ ]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(28)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    sf = downsample
    for ch,ch_label in enumerate(channels):
        features = {}
        features['center'] = 'california'
        features['subject'] = ('c_' + subject)
        features['channel'] = ch_label
        features['delta'] = bandpower(epochs.get_data()[:,ch,:], sf, [1, 4], 'multitaper_average', relative=True)
        features['theta'] = bandpower(epochs.get_data()[:,ch,:], sf, [4, 8], 'multitaper_average', relative=True)
        features['alpha'] = bandpower(epochs.get_data()[:,ch,:], sf, [8, 13], 'multitaper_average', relative=True)
        features['beta'] = bandpower(epochs.get_data()[:,ch,:], sf, [13, 30], 'multitaper_average', relative=True)
        features['pre_alpha'] = bandpower(epochs.get_data()[:,ch,:], sf, [5.5, 8], 'multitaper_average', relative=True)
        features['slow_theta'] = bandpower(epochs.get_data()[:,ch,:], sf, [4, 5.5], 'multitaper_average', relative=True)
        features['alpha_theta'] = bandpower(epochs.get_data()[:,ch,:], sf, [8, 13], 'multitaper_average', relative=True) / bandpower(epochs.get_data()[:,ch,:], sf, [4, 8], 'multitaper_average', relative=True)
        dict_list.append(features)
bandpowers = pd.DataFrame(dict_list)

bandpowers.to_csv('D:/Papers/FDA_spectral/review/reanalysis/california/bandpowers_averaged.csv', index=False)

Wall time: 28min 51s


FDA Approach

In [ ]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(28)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    sf = downsample
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            features = {}
            features['center'] = 'california'
            features['subject'] = ('c_' + subject)
            features['channel'] = ch_label
            features['epoch'] = ep
            features['delta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [1, 4], 'multitaper', relative=True)
            features['theta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [4, 8], 'multitaper', relative=True)
            features['alpha'] = bandpower(epochs.get_data()[ep,ch,:], sf, [8, 13], 'multitaper', relative=True)
            features['beta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [13, 30], 'multitaper', relative=True)
            features['pre_alpha'] = bandpower(epochs.get_data()[ep,ch,:], sf, [5.5, 8], 'multitaper', relative=True)
            features['slow_theta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [4, 5.5], 'multitaper', relative=True)
            features['alpha_theta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [8, 13], 'multitaper', relative=True) / bandpower(epochs.get_data()[ep,ch,:], sf, [4, 8], 'multitaper', relative=True)
            dict_list.append(features)
bandpowers_fda = pd.DataFrame(dict_list)

bandpowers_fda.to_csv('D:/Papers/FDA_spectral/review/reanalysis/california/bandpowers_fda.csv', index=False)

Wall time: 1h 24min 58s


##### Iowa

In [ ]:
iowa={
'layout':{'extension':'.fif', 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['T8', 'CP3', 'T7', 'C4', 'C5', 'FT8', 'F6', 'FC5', 'FC3', 'C2', 'TP9', 'F7', 'C6', 'CP6', 'F2', 'TP7', 'PO7', 'FCZ', 'O2', 'P2', 'CZ', 'F1', 'P6', 'AF8', 'C3', 'AF4', 'CPZ', 'F5', 'CP1', 'O1', 'P8', 'FC6', 'FC2', 'P4', 'F8', 'FP2', 'AF3', 'POZ', 'FT10', 'P1', 'FC1', 'TP10', 'FP1', 'AF7', 'PO8', 'TP8', 'F4', 'FZ', 'AFZ', 'OZ', 'CP4', 'P3', 'FT7', 'CP5', 'P5', 'P7', 'F3', 'FC4', 'CP2', 'C1'],
    'path':'D:/EEGs/PhD_datasets/BIDS/iowa/'
}

#Path of the BIDS folder
bids_root = 'D:/EEGs/PhD_datasets/BIDS/iowa/'
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=iowa #DEFINE DATASET

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]

print(len(eegs), eegs)

28 ['D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-001\\eeg\\sub-001_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-002\\eeg\\sub-002_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-003\\eeg\\sub-003_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-004\\eeg\\sub-004_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-005\\eeg\\sub-005_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-006\\eeg\\sub-006_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-007\\eeg\\sub-007_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-008\\eeg\\sub-008_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmon

Averaged epochs approach

In [ ]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(16)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    sf = downsample
    for ch,ch_label in enumerate(channels):
        features = {}
        features['center'] = 'iowa'
        features['subject'] = ('i_' + subject)
        features['channel'] = ch_label
        features['delta'] = bandpower(epochs.get_data()[:,ch,:], sf, [1, 4], 'multitaper_average', relative=True)
        features['theta'] = bandpower(epochs.get_data()[:,ch,:], sf, [4, 8], 'multitaper_average', relative=True)
        features['alpha'] = bandpower(epochs.get_data()[:,ch,:], sf, [8, 13], 'multitaper_average', relative=True)
        features['beta'] = bandpower(epochs.get_data()[:,ch,:], sf, [13, 30], 'multitaper_average', relative=True)
        features['pre_alpha'] = bandpower(epochs.get_data()[:,ch,:], sf, [5.5, 8], 'multitaper_average', relative=True)
        features['slow_theta'] = bandpower(epochs.get_data()[:,ch,:], sf, [4, 5.5], 'multitaper_average', relative=True)
        features['alpha_theta'] = bandpower(epochs.get_data()[:,ch,:], sf, [8, 13], 'multitaper_average', relative=True) / bandpower(epochs.get_data()[:,ch,:], sf, [4, 8], 'multitaper_average', relative=True)
        dict_list.append(features)
bandpowers = pd.DataFrame(dict_list)

bandpowers.to_csv('D:/Papers/FDA_spectral/review/reanalysis/iowa/bandpowers_averaged.csv', index=False)

Wall time: 41min 56s


FDA Approach

In [ ]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(16)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    sf = downsample
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            features = {}
            features['center'] = 'iowa'
            features['subject'] = ('i_' + subject)
            features['channel'] = ch_label
            features['epoch'] = ep
            features['delta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [1, 4], 'multitaper', relative=True)
            features['theta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [4, 8], 'multitaper', relative=True)
            features['alpha'] = bandpower(epochs.get_data()[ep,ch,:], sf, [8, 13], 'multitaper', relative=True)
            features['beta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [13, 30], 'multitaper', relative=True)
            features['pre_alpha'] = bandpower(epochs.get_data()[ep,ch,:], sf, [5.5, 8], 'multitaper', relative=True)
            features['slow_theta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [4, 5.5], 'multitaper', relative=True)
            features['alpha_theta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [8, 13], 'multitaper', relative=True) / bandpower(epochs.get_data()[ep,ch,:], sf, [4, 8], 'multitaper', relative=True)
            dict_list.append(features)
bandpowers_fda = pd.DataFrame(dict_list)

bandpowers_fda.to_csv('D:/Papers/FDA_spectral/review/reanalysis/iowa/bandpowers_fda.csv', index=False)

Wall time: 1h 32min 50s


##### Finland ####

In [ ]:
finland={
'layout':{'extension':'.fif', 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'FZ', 'CZ', 'PZ', 'IZ', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'FT9', 'FT10', 'TP9', 'TP10', 'F1', 'F2', 'C1', 'C2', 'P1', 'P2', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'AF7', 'AF8', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'FPZ', 'CPZ', 'POZ', 'OZ'],
    'path':'D:/EEGs/PhD_datasets/BIDS/finland/'
}

#Path of the BIDS folder
bids_root = 'D:/EEGs/PhD_datasets/BIDS/finland/'
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=finland #DEFINE DATASET

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]

print(len(eegs), eegs)

38 ['D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-001\\eeg\\sub-001_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-002\\eeg\\sub-002_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-003\\eeg\\sub-003_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-004\\eeg\\sub-004_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-005\\eeg\\sub-005_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-006\\eeg\\sub-006_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-007\\eeg\\sub-007_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-008\\eeg\\sub-008_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finla

Averaged epochs approach

In [ ]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(18)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    sf = downsample
    for ch,ch_label in enumerate(channels):
        features = {}
        features['center'] = 'finland'
        features['subject'] = ('f_' + subject)
        features['channel'] = ch_label
        features['delta'] = bandpower(epochs.get_data()[:,ch,:], sf, [1, 4], 'multitaper_average', relative=True)
        features['theta'] = bandpower(epochs.get_data()[:,ch,:], sf, [4, 8], 'multitaper_average', relative=True)
        features['alpha'] = bandpower(epochs.get_data()[:,ch,:], sf, [8, 13], 'multitaper_average', relative=True)
        features['beta'] = bandpower(epochs.get_data()[:,ch,:], sf, [13, 30], 'multitaper_average', relative=True)
        features['pre_alpha'] = bandpower(epochs.get_data()[:,ch,:], sf, [5.5, 8], 'multitaper_average', relative=True)
        features['slow_theta'] = bandpower(epochs.get_data()[:,ch,:], sf, [4, 5.5], 'multitaper_average', relative=True)
        features['alpha_theta'] = bandpower(epochs.get_data()[:,ch,:], sf, [8, 13], 'multitaper_average', relative=True) / bandpower(epochs.get_data()[:,ch,:], sf, [4, 8], 'multitaper_average', relative=True)
        dict_list.append(features)
bandpowers = pd.DataFrame(dict_list)

bandpowers.to_csv('D:/Papers/FDA_spectral/review/reanalysis/finland/bandpowers_averaged.csv', index=False)

Wall time: 40min 29s


FDA Approach

In [ ]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(18)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    sf = downsample
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            features = {}
            features['center'] = 'finland'
            features['subject'] = ('f_' + subject)
            features['channel'] = ch_label
            features['epoch'] = ep
            features['delta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [1, 4], 'multitaper', relative=True)
            features['theta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [4, 8], 'multitaper', relative=True)
            features['alpha'] = bandpower(epochs.get_data()[ep,ch,:], sf, [8, 13], 'multitaper', relative=True)
            features['beta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [13, 30], 'multitaper', relative=True)
            features['pre_alpha'] = bandpower(epochs.get_data()[ep,ch,:], sf, [5.5, 8], 'multitaper', relative=True)
            features['slow_theta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [4, 5.5], 'multitaper', relative=True)
            features['alpha_theta'] = bandpower(epochs.get_data()[ep,ch,:], sf, [8, 13], 'multitaper', relative=True) / bandpower(epochs.get_data()[ep,ch,:], sf, [4, 8], 'multitaper', relative=True)
            dict_list.append(features)
bandpowers_fda = pd.DataFrame(dict_list)

bandpowers_fda.to_csv('D:/Papers/FDA_spectral/review/reanalysis/finland/bandpowers_fda.csv', index=False)

Wall time: 1h 49min 6s


#### Entropy

In [16]:
import antropy as ant

##### Medellin

In [103]:
antioquia={
'layout':{'extension':'.fif', 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'O1', 'OZ', 'O2'],
    'path':'D:/EEGs/PhD_datasets/BIDS/antioquia/'
}

#Path of the BIDS folder
bids_root = 'D:/EEGs/PhD_datasets/BIDS/antioquia/'
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=antioquia

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]
print(len(eegs), eegs)

72 ['D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-001\\eeg\\sub-001_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-002\\eeg\\sub-002_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-003\\eeg\\sub-003_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-004\\eeg\\sub-004_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-005\\eeg\\sub-005_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-006\\eeg\\sub-006_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-007\\eeg\\sub-007_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-008\\eeg\\sub-008_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datas

FDA approach (epoch-to-epoch) & then Averaged across epochs

In [104]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(44)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    sf = downsample
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            features = {}
            features['center'] = 'medellin'
            features['subject'] = ('m_' + subject)
            features['channel'] = ch_label
            features['epoch'] = ep
            features['permutation'] = ant.perm_entropy(epochs.get_data()[ep,ch,:], normalize=True)
            features['sample'] = ant.sample_entropy(epochs.get_data()[ep,ch,:])
            features['approximate'] = ant.app_entropy(epochs.get_data()[ep,ch,:])
            features['svd_ent'] = ant.svd_entropy(epochs.get_data()[ep,ch,:], order=5, normalize=True)             
            features['higuchi_fd']  = ant.higuchi_fd(epochs.get_data()[ep,ch,:])  
            features['hjort_mobility']  = (ant.hjorth_params(epochs.get_data()[ep,ch,:]))[0]
            features['hjort_complexity']  = (ant.hjorth_params(epochs.get_data()[ep,ch,:]))[1]
            features['detrended_fluct']  = ant.detrended_fluctuation(epochs.get_data()[ep,ch,:])
            features['katz_fd']  = ant.katz_fd(epochs.get_data()[ep,ch,:])
            features['petrosian_fd']  = ant.petrosian_fd(epochs.get_data()[ep,ch,:])
            dict_list.append(features)
entropies = pd.DataFrame(dict_list)

entropies.to_csv('D:/Papers/FDA_spectral/review/reanalysis/medellin/entropies_fda.csv', index=False)

permutation = pd.DataFrame(entropies.groupby(['center','subject', 'channel']).permutation.mean())
entropies.rename(columns = {'sample':'sample_ent'}, inplace = True)
sample_ent = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).sample_ent.mean())
approximate = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).approximate.mean())
svd_ent = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).svd_ent.mean())
higuchi_fd = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).higuchi_fd.mean())
hjort_mobility = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).hjort_mobility.mean())
hjort_complexity = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).hjort_complexity.mean())
detrended_fluct = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).detrended_fluct.mean())
katz_fd = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).katz_fd.mean())
petrosian_fd = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).petrosian_fd.mean())

dfs = [permutation, sample_ent, approximate, svd_ent, higuchi_fd, hjort_mobility, hjort_complexity, detrended_fluct, katz_fd, petrosian_fd]
entropies_ave = pd.DataFrame(pd.concat(dfs, axis = 1))
entropies_ave.to_csv('D:/Papers/FDA_spectral/review/reanalysis/medellin/entropies_averaged.csv', index=['center', 'subject', 'channel'])

Wall time: 2h 36min 47s


##### California #####

In [16]:
california={
'layout':{'extension':'.fif', 'session':['hc', 'on'], 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['FP1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5', 'P7', 'P3', 'PZ', 'PO3', 'O1', 'OZ', 'O2', 'PO4', 'P4', 'P8', 'CP6', 'CP2', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'AF4', 'FP2', 'FZ', 'CZ'],
    'path':'D:/EEGs/PhD_datasets/BIDS/california/'
}

#Path of the BIDS folder
bids_root = 'D:/EEGs/PhD_datasets/BIDS/california/'
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=california #DEFINE DATASET

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]

print(len(eegs), eegs)

31 ['D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc1\\ses-hc\\eeg\\sub-hc1_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc2\\ses-hc\\eeg\\sub-hc2_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc4\\ses-hc\\eeg\\sub-hc4_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc7\\ses-hc\\eeg\\sub-hc7_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc8\\ses-hc\\eeg\\sub-hc8_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc10\\ses-hc\\eeg\\sub-hc10_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc18\\ses-hc\\eeg\\sub-hc18_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\B

FDA approach (epoch-to-epoch) & then Averaged across epochs

In [109]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(28)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    sf = downsample
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            features = {}
            features['center'] = 'california'
            features['subject'] = ('c_' + subject)
            features['channel'] = ch_label
            features['epoch'] = ep
            features['permutation'] = ant.perm_entropy(epochs.get_data()[ep,ch,:], normalize=True)
            features['sample'] = ant.sample_entropy(epochs.get_data()[ep,ch,:])
            features['approximate'] = ant.app_entropy(epochs.get_data()[ep,ch,:])
            features['svd_ent'] = ant.svd_entropy(epochs.get_data()[ep,ch,:], order=5, normalize=True)             
            features['higuchi_fd']  = ant.higuchi_fd(epochs.get_data()[ep,ch,:])  
            features['hjort_mobility']  = (ant.hjorth_params(epochs.get_data()[ep,ch,:]))[0]
            features['hjort_complexity']  = (ant.hjorth_params(epochs.get_data()[ep,ch,:]))[1]
            features['detrended_fluct']  = ant.detrended_fluctuation(epochs.get_data()[ep,ch,:])
            features['katz_fd']  = ant.katz_fd(epochs.get_data()[ep,ch,:])
            features['petrosian_fd']  = ant.petrosian_fd(epochs.get_data()[ep,ch,:])
            dict_list.append(features)
entropies = pd.DataFrame(dict_list)

entropies.to_csv('D:/Papers/FDA_spectral/review/reanalysis/california/entropies_fda.csv', index=False)

permutation = pd.DataFrame(entropies.groupby(['center','subject', 'channel']).permutation.mean())
entropies.rename(columns = {'sample':'sample_ent'}, inplace = True)
sample_ent = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).sample_ent.mean())
approximate = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).approximate.mean())
svd_ent = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).svd_ent.mean())
higuchi_fd = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).higuchi_fd.mean())
hjort_mobility = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).hjort_mobility.mean())
hjort_complexity = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).hjort_complexity.mean())
detrended_fluct = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).detrended_fluct.mean())
katz_fd = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).katz_fd.mean())
petrosian_fd = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).petrosian_fd.mean())

dfs = [permutation, sample_ent, approximate, svd_ent, higuchi_fd, hjort_mobility, hjort_complexity, detrended_fluct, katz_fd, petrosian_fd]
entropies_ave = pd.DataFrame(pd.concat(dfs, axis = 1))
entropies_ave.to_csv('D:/Papers/FDA_spectral/review/reanalysis/california/entropies_averaged.csv', index=['center', 'subject', 'channel'])

Wall time: 24min 4s


##### Iowa

In [110]:
iowa={
'layout':{'extension':'.fif', 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['T8', 'CP3', 'T7', 'C4', 'C5', 'FT8', 'F6', 'FC5', 'FC3', 'C2', 'TP9', 'F7', 'C6', 'CP6', 'F2', 'TP7', 'PO7', 'FCZ', 'O2', 'P2', 'CZ', 'F1', 'P6', 'AF8', 'C3', 'AF4', 'CPZ', 'F5', 'CP1', 'O1', 'P8', 'FC6', 'FC2', 'P4', 'F8', 'FP2', 'AF3', 'POZ', 'FT10', 'P1', 'FC1', 'TP10', 'FP1', 'AF7', 'PO8', 'TP8', 'F4', 'FZ', 'AFZ', 'OZ', 'CP4', 'P3', 'FT7', 'CP5', 'P5', 'P7', 'F3', 'FC4', 'CP2', 'C1'],
    'path':'D:/EEGs/PhD_datasets/BIDS/iowa/'
}

#Path of the BIDS folder
bids_root = 'D:/EEGs/PhD_datasets/BIDS/iowa/'
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=iowa #DEFINE DATASET

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]

print(len(eegs), eegs)

28 ['D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-001\\eeg\\sub-001_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-002\\eeg\\sub-002_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-003\\eeg\\sub-003_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-004\\eeg\\sub-004_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-005\\eeg\\sub-005_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-006\\eeg\\sub-006_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-007\\eeg\\sub-007_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-008\\eeg\\sub-008_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmon

FDA approach (epoch-to-epoch) & then Averaged across epochs

In [111]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(16)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    sf = downsample
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            features = {}
            features['center'] = 'iowa'
            features['subject'] = ('i_' + subject)
            features['channel'] = ch_label
            features['epoch'] = ep
            features['permutation'] = ant.perm_entropy(epochs.get_data()[ep,ch,:], normalize=True)
            features['sample'] = ant.sample_entropy(epochs.get_data()[ep,ch,:])
            features['approximate'] = ant.app_entropy(epochs.get_data()[ep,ch,:])
            features['svd_ent'] = ant.svd_entropy(epochs.get_data()[ep,ch,:], order=5, normalize=True)             
            features['higuchi_fd']  = ant.higuchi_fd(epochs.get_data()[ep,ch,:])  
            features['hjort_mobility']  = (ant.hjorth_params(epochs.get_data()[ep,ch,:]))[0]
            features['hjort_complexity']  = (ant.hjorth_params(epochs.get_data()[ep,ch,:]))[1]
            features['detrended_fluct']  = ant.detrended_fluctuation(epochs.get_data()[ep,ch,:])
            features['katz_fd']  = ant.katz_fd(epochs.get_data()[ep,ch,:])
            features['petrosian_fd']  = ant.petrosian_fd(epochs.get_data()[ep,ch,:])
            dict_list.append(features)
entropies = pd.DataFrame(dict_list)

entropies.to_csv('D:/Papers/FDA_spectral/review/reanalysis/iowa/entropies_fda.csv', index=False)

permutation = pd.DataFrame(entropies.groupby(['center','subject', 'channel']).permutation.mean())
entropies.rename(columns = {'sample':'sample_ent'}, inplace = True)
sample_ent = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).sample_ent.mean())
approximate = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).approximate.mean())
svd_ent = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).svd_ent.mean())
higuchi_fd = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).higuchi_fd.mean())
hjort_mobility = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).hjort_mobility.mean())
hjort_complexity = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).hjort_complexity.mean())
detrended_fluct = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).detrended_fluct.mean())
katz_fd = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).katz_fd.mean())
petrosian_fd = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).petrosian_fd.mean())

dfs = [permutation, sample_ent, approximate, svd_ent, higuchi_fd, hjort_mobility, hjort_complexity, detrended_fluct, katz_fd, petrosian_fd]
entropies_ave = pd.DataFrame(pd.concat(dfs, axis = 1))
entropies_ave.to_csv('D:/Papers/FDA_spectral/review/reanalysis/iowa/entropies_averaged.csv', index=['center', 'subject', 'channel'])

Wall time: 23min 16s


##### Finland ####

In [106]:
finland={
'layout':{'extension':'.fif', 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'FZ', 'CZ', 'PZ', 'IZ', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'FT9', 'FT10', 'TP9', 'TP10', 'F1', 'F2', 'C1', 'C2', 'P1', 'P2', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'AF7', 'AF8', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'FPZ', 'CPZ', 'POZ', 'OZ'],
    'path':'D:/EEGs/PhD_datasets/BIDS/finland/'
}

#Path of the BIDS folder
bids_root = 'D:/EEGs/PhD_datasets/BIDS/finland/'
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=finland #DEFINE DATASET

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]

print(len(eegs), eegs)

38 ['D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-001\\eeg\\sub-001_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-002\\eeg\\sub-002_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-003\\eeg\\sub-003_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-004\\eeg\\sub-004_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-005\\eeg\\sub-005_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-006\\eeg\\sub-006_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-007\\eeg\\sub-007_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-008\\eeg\\sub-008_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finla

FDA approach (epoch-to-epoch) & then Averaged across epochs

In [107]:
%%time
%%capture
dict_list = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(18)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    sf = downsample
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            features = {}
            features['center'] = 'finland'
            features['subject'] = ('f_' + subject)
            features['channel'] = ch_label
            features['epoch'] = ep
            features['permutation'] = ant.perm_entropy(epochs.get_data()[ep,ch,:], normalize=True)
            features['sample'] = ant.sample_entropy(epochs.get_data()[ep,ch,:])
            features['approximate'] = ant.app_entropy(epochs.get_data()[ep,ch,:])
            features['svd_ent'] = ant.svd_entropy(epochs.get_data()[ep,ch,:], order=5, normalize=True)             
            features['higuchi_fd']  = ant.higuchi_fd(epochs.get_data()[ep,ch,:])  
            features['hjort_mobility']  = (ant.hjorth_params(epochs.get_data()[ep,ch,:]))[0]
            features['hjort_complexity']  = (ant.hjorth_params(epochs.get_data()[ep,ch,:]))[1]
            features['detrended_fluct']  = ant.detrended_fluctuation(epochs.get_data()[ep,ch,:])
            features['katz_fd']  = ant.katz_fd(epochs.get_data()[ep,ch,:])
            features['petrosian_fd']  = ant.petrosian_fd(epochs.get_data()[ep,ch,:])
            dict_list.append(features)
entropies = pd.DataFrame(dict_list)

entropies.to_csv('D:/Papers/FDA_spectral/review/reanalysis/finland/entropies_fda.csv', index=False)

permutation = pd.DataFrame(entropies.groupby(['center','subject', 'channel']).permutation.mean())
entropies.rename(columns = {'sample':'sample_ent'}, inplace = True)
sample_ent = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).sample_ent.mean())
approximate = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).approximate.mean())
svd_ent = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).svd_ent.mean())
higuchi_fd = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).higuchi_fd.mean())
hjort_mobility = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).hjort_mobility.mean())
hjort_complexity = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).hjort_complexity.mean())
detrended_fluct = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).detrended_fluct.mean())
katz_fd = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).katz_fd.mean())
petrosian_fd = pd.DataFrame(entropies.groupby(['center', 'subject', 'channel']).petrosian_fd.mean())

dfs = [permutation, sample_ent, approximate, svd_ent, higuchi_fd, hjort_mobility, hjort_complexity, detrended_fluct, katz_fd, petrosian_fd]
entropies_ave = pd.DataFrame(pd.concat(dfs, axis = 1))
entropies_ave.to_csv('D:/Papers/FDA_spectral/review/reanalysis/finland/entropies_averaged.csv', index=['center', 'subject', 'channel'])

Wall time: 37min 24s


#### Merge averaged epochs data

Medellin

In [241]:
# Read the Dataset 1 (medellin) participants.tsv metadata and merge demographic variables with features (for ComBat control of biological covariates)

metadata_cols = ['participant_id', 'group' , 'age', 'gender'] # names of the metadata columns to be merged

participants_file = r"D:\EEGs\PhD_datasets\BIDS\antioquia\participants.tsv"
participants_df = pd.read_csv(participants_file, sep="\t", converters = {'subject': str})
participants_df.columns= participants_df.columns.str.lower() # To set all the column names in lower case
participants_df['participant_id'] = participants_df['participant_id'].str.replace('sub-', 'm_')  # Drop the "sub-" prefix used for BIDS


entropies_file = r"D:/Papers/FDA_spectral/review/reanalysis/medellin/entropies_averaged.csv"
entropies_df = pd.read_csv(entropies_file, sep=",", converters = {'subject': str})
entropies_df.rename(columns = {'subject':'participant_id'}, inplace = True)
df_1 = pd.DataFrame(pd.merge(participants_df[metadata_cols], entropies_df, on="participant_id")) # Merge metadata columns with the features into a new data frame
df_1['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (f_ for medellin)


bandpowers_file = r"D:/Papers/FDA_spectral/review/reanalysis/medellin/bandpowers_averaged.csv"
bandpowers_df = pd.read_csv(bandpowers_file, sep=",", converters = {'subject': str})
bandpowers_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_2 = pd.DataFrame(pd.merge(df_1, bandpowers_df, on=["participant_id", 'channel', 'center'])) # Merge metadata columns with the features into a new data frame
df_2['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for medellin)

df_file = r"D:/Papers/FDA_spectral/review/reanalysis/medellin/df_averaged.csv"
df_df = pd.read_csv(df_file, sep=",", converters = {'subject': str})
df_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_3 = pd.DataFrame(pd.merge(df_2, df_df, on=["participant_id", 'channel', 'center'])) # Merge metadata columns with the features into a new data frame
df_3['participant_id'] = '' + df_2['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for medellin)

medellin_df = df_3

medellin_df.to_csv('D:/Papers/FDA_spectral/review/reanalysis/medellin/medellin_averaged.csv', index=False)

California

In [242]:
# Read the Dataset 1 (california) participants.tsv metadata and merge demographic variables with features (for ComBat control of biological covariates)

participants_file = r"D:\EEGs\PhD_datasets\BIDS\california\participants.tsv"
metadata_cols = ['participant_id', 'group','age', 'gender']

participants_df = pd.read_csv(participants_file, sep="\t")
participants_df['participant_id'] = participants_df['participant_id'].str.replace('sub-', 'c_')  # Drop the "sub-" prefix used for BIDS

entropies_file = r"D:/Papers/FDA_spectral/review/reanalysis/california/entropies_averaged.csv"
entropies_df = pd.read_csv(entropies_file, sep=",", converters = {'subject': str})
entropies_df.rename(columns = {'subject':'participant_id'}, inplace = True)
df_1 = pd.DataFrame(pd.merge(participants_df[metadata_cols], entropies_df, on="participant_id")) # Merge metadata columns with the features into a new data frame
df_1['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (f_ for california)


bandpowers_file = r"D:/Papers/FDA_spectral/review/reanalysis/california/bandpowers_averaged.csv"
bandpowers_df = pd.read_csv(bandpowers_file, sep=",", converters = {'subject': str})
bandpowers_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_2 = pd.DataFrame(pd.merge(df_1, bandpowers_df, on=["participant_id", 'channel', 'center'])) # Merge metadata columns with the features into a new data frame
df_2['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for california)

df_file = r"D:/Papers/FDA_spectral/review/reanalysis/california/df_averaged.csv"
df_df = pd.read_csv(df_file, sep=",", converters = {'subject': str})
df_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_3 = pd.DataFrame(pd.merge(df_2, df_df, on=["participant_id", 'channel', 'center'])) # Merge metadata columns with the features into a new data frame
df_3['participant_id'] = '' + df_2['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for california)

california_df = df_3

california_df.to_csv('D:/Papers/FDA_spectral/review/reanalysis/california/california_averaged.csv', index=False)

Iowa

In [243]:
# Read the Dataset 1 (iowa) participants.tsv metadata and merge demographic variables with features (for ComBat control of biological covariates)

metadata_cols = ['participant_id', 'group' , 'age', 'gender'] # names of the metadata columns to be merged

participants_file = r"D:\EEGs\PhD_datasets\BIDS\iowa\participants.tsv"
participants_df = pd.read_csv(participants_file, sep="\t", converters = {'subject': str})
participants_df.columns= participants_df.columns.str.lower() # To set all the column names in lower case
participants_df['participant_id'] = participants_df['participant_id'].str.replace('sub-', 'i_')  # Drop the "sub-" prefix used for BIDS


entropies_file = r"D:/Papers/FDA_spectral/review/reanalysis/iowa/entropies_averaged.csv"
entropies_df = pd.read_csv(entropies_file, sep=",", converters = {'subject': str})
entropies_df.rename(columns = {'subject':'participant_id'}, inplace = True)
df_1 = pd.DataFrame(pd.merge(participants_df[metadata_cols], entropies_df, on="participant_id")) # Merge metadata columns with the features into a new data frame
df_1['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (f_ for iowa)


bandpowers_file = r"D:/Papers/FDA_spectral/review/reanalysis/iowa/bandpowers_averaged.csv"
bandpowers_df = pd.read_csv(bandpowers_file, sep=",", converters = {'subject': str})
bandpowers_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_2 = pd.DataFrame(pd.merge(df_1, bandpowers_df, on=["participant_id", 'channel', 'center'])) # Merge metadata columns with the features into a new data frame
df_2['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for iowa)

df_file = r"D:/Papers/FDA_spectral/review/reanalysis/iowa/df_averaged.csv"
df_df = pd.read_csv(df_file, sep=",", converters = {'subject': str})
df_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_3 = pd.DataFrame(pd.merge(df_2, df_df, on=["participant_id", 'channel', 'center'])) # Merge metadata columns with the features into a new data frame
df_3['participant_id'] = '' + df_2['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for iowa)

iowa_df = df_3

iowa_df.to_csv('D:/Papers/FDA_spectral/review/reanalysis/iowa/iowa_averaged.csv', index=False)

Finland

In [244]:
# Read the Dataset 1 (Finland) participants.tsv metadata and merge demographic variables with features (for ComBat control of biological covariates)

metadata_cols = ['participant_id', 'group' , 'age', 'gender'] # names of the metadata columns to be merged

participants_file = r"D:\EEGs\PhD_datasets\BIDS\finland\participants.tsv"
participants_df = pd.read_csv(participants_file, sep="\t", converters = {'subject': str})
participants_df.columns= participants_df.columns.str.lower() # To set all the column names in lower case
participants_df['participant_id'] = participants_df['participant_id'].str.replace('sub-', 'f_')  # Drop the "sub-" prefix used for BIDS


entropies_file = r"D:/Papers/FDA_spectral/review/reanalysis/finland/entropies_averaged.csv"
entropies_df = pd.read_csv(entropies_file, sep=",", converters = {'subject': str})
entropies_df.rename(columns = {'subject':'participant_id'}, inplace = True)
df_1 = pd.DataFrame(pd.merge(participants_df[metadata_cols], entropies_df, on="participant_id")) # Merge metadata columns with the features into a new data frame
df_1['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (f_ for Finland)


bandpowers_file = r"D:/Papers/FDA_spectral/review/reanalysis/finland/bandpowers_averaged.csv"
bandpowers_df = pd.read_csv(bandpowers_file, sep=",", converters = {'subject': str})
bandpowers_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_2 = pd.DataFrame(pd.merge(df_1, bandpowers_df, on=["participant_id", 'channel', 'center'])) # Merge metadata columns with the features into a new data frame
df_2['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for Finland)

df_file = r"D:/Papers/FDA_spectral/review/reanalysis/finland/df_averaged.csv"
df_df = pd.read_csv(df_file, sep=",", converters = {'subject': str})
df_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_3 = pd.DataFrame(pd.merge(df_2, df_df, on=["participant_id", 'channel', 'center'])) # Merge metadata columns with the features into a new data frame
df_3['participant_id'] = '' + df_2['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for Finland)

finland_df = df_3

finland_df.to_csv('D:/Papers/FDA_spectral/review/reanalysis/finland/finland_averaged.csv', index=False)

In [245]:
all_centers = pd.concat([medellin_df, california_df, iowa_df, finland_df], axis=0)
all_centers.to_csv('D:/Papers/FDA_spectral/review/reanalysis/allcenters_averaged.csv', index=False)

#### Merge fda data

Medellin

In [246]:
# Read the Dataset 1 (medellin) participants.tsv metadata and merge demographic variables with features (for ComBat control of biological covariates)

metadata_cols = ['participant_id', 'group' , 'age', 'gender'] # names of the metadata columns to be merged

participants_file = r"D:\EEGs\PhD_datasets\BIDS\antioquia\participants.tsv"
participants_df = pd.read_csv(participants_file, sep="\t", converters = {'subject': str})
participants_df.columns= participants_df.columns.str.lower() # To set all the column names in lower case
participants_df['participant_id'] = participants_df['participant_id'].str.replace('sub-', 'm_')  # Drop the "sub-" prefix used for BIDS


entropies_file = r"D:/Papers/FDA_spectral/review/reanalysis/medellin/entropies_fda.csv"
entropies_df = pd.read_csv(entropies_file, sep=",", converters = {'subject': str})
entropies_df.rename(columns = {'subject':'participant_id'}, inplace = True)
df_1 = pd.DataFrame(pd.merge(participants_df[metadata_cols], entropies_df, on="participant_id")) # Merge metadata columns with the features into a new data frame
df_1['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (f_ for medellin)


bandpowers_file = r"D:/Papers/FDA_spectral/review/reanalysis/medellin/bandpowers_fda.csv"
bandpowers_df = pd.read_csv(bandpowers_file, sep=",", converters = {'subject': str})
bandpowers_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_2 = pd.DataFrame(pd.merge(df_1, bandpowers_df, on=["participant_id", 'channel', 'center', 'epoch'])) # Merge metadata columns with the features into a new data frame
df_2['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for medellin)

df_file = r"D:/Papers/FDA_spectral/review/reanalysis/medellin/df_fda.csv"
df_df = pd.read_csv(df_file, sep=",", converters = {'subject': str})
df_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_3 = pd.DataFrame(pd.merge(df_2, df_df, on=["participant_id", 'channel', 'center', 'epoch'])) # Merge metadata columns with the features into a new data frame
df_3['participant_id'] = '' + df_2['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for medellin)

medellin_df = df_3

medellin_df.to_csv('D:/Papers/FDA_spectral/review/reanalysis/medellin/medellin_fda.csv', index=False)

California

In [247]:
# Read the Dataset 1 (california) participants.tsv metadata and merge demographic variables with features (for ComBat control of biological covariates)


participants_file = r"D:\EEGs\PhD_datasets\BIDS\california\participants.tsv"
metadata_cols = ['participant_id', 'group','age', 'gender']

participants_df = pd.read_csv(participants_file, sep="\t")
participants_df['participant_id'] = participants_df['participant_id'].str.replace('sub-', 'c_')  # Drop the "sub-" prefix used for BIDS


entropies_file = r"D:/Papers/FDA_spectral/review/reanalysis/california/entropies_fda.csv"
entropies_df = pd.read_csv(entropies_file, sep=",", converters = {'subject': str})
entropies_df.rename(columns = {'subject':'participant_id'}, inplace = True)
df_1 = pd.DataFrame(pd.merge(participants_df[metadata_cols], entropies_df, on="participant_id")) # Merge metadata columns with the features into a new data frame
df_1['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (f_ for california)


bandpowers_file = r"D:/Papers/FDA_spectral/review/reanalysis/california/bandpowers_fda.csv"
bandpowers_df = pd.read_csv(bandpowers_file, sep=",", converters = {'subject': str})
bandpowers_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_2 = pd.DataFrame(pd.merge(df_1, bandpowers_df, on=["participant_id", 'channel', 'center', 'epoch'])) # Merge metadata columns with the features into a new data frame
df_2['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for california)

df_file = r"D:/Papers/FDA_spectral/review/reanalysis/california/df_fda.csv"
df_df = pd.read_csv(df_file, sep=",", converters = {'subject': str})
df_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_3 = pd.DataFrame(pd.merge(df_2, df_df, on=["participant_id", 'channel', 'center', 'epoch'])) # Merge metadata columns with the features into a new data frame
df_3['participant_id'] = '' + df_2['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for california)

california_df = df_3

california_df.to_csv('D:/Papers/FDA_spectral/review/reanalysis/california/california_fda.csv', index=False)

Iowa

In [248]:
# Read the Dataset 1 (iowa) participants.tsv metadata and merge demographic variables with features (for ComBat control of biological covariates)

metadata_cols = ['participant_id', 'group' , 'age', 'gender'] # names of the metadata columns to be merged

participants_file = r"D:\EEGs\PhD_datasets\BIDS\iowa\participants.tsv"
participants_df = pd.read_csv(participants_file, sep="\t", converters = {'subject': str})
participants_df.columns= participants_df.columns.str.lower() # To set all the column names in lower case
participants_df['participant_id'] = participants_df['participant_id'].str.replace('sub-', 'i_')  # Drop the "sub-" prefix used for BIDS


entropies_file = r"D:/Papers/FDA_spectral/review/reanalysis/iowa/entropies_fda.csv"
entropies_df = pd.read_csv(entropies_file, sep=",", converters = {'subject': str})
entropies_df.rename(columns = {'subject':'participant_id'}, inplace = True)
df_1 = pd.DataFrame(pd.merge(participants_df[metadata_cols], entropies_df, on="participant_id")) # Merge metadata columns with the features into a new data frame
df_1['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (f_ for iowa)


bandpowers_file = r"D:/Papers/FDA_spectral/review/reanalysis/iowa/bandpowers_fda.csv"
bandpowers_df = pd.read_csv(bandpowers_file, sep=",", converters = {'subject': str})
bandpowers_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_2 = pd.DataFrame(pd.merge(df_1, bandpowers_df, on=["participant_id", 'channel', 'center', 'epoch'])) # Merge metadata columns with the features into a new data frame
df_2['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for iowa)

df_file = r"D:/Papers/FDA_spectral/review/reanalysis/iowa/df_fda.csv"
df_df = pd.read_csv(df_file, sep=",", converters = {'subject': str})
df_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_3 = pd.DataFrame(pd.merge(df_2, df_df, on=["participant_id", 'channel', 'center', 'epoch'])) # Merge metadata columns with the features into a new data frame
df_3['participant_id'] = '' + df_2['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for iowa)

iowa_df = df_3

iowa_df.to_csv('D:/Papers/FDA_spectral/review/reanalysis/iowa/iowa_fda.csv', index=False)

Finland

In [249]:
# Read the Dataset 1 (Finland) participants.tsv metadata and merge demographic variables with features (for ComBat control of biological covariates)

metadata_cols = ['participant_id', 'group' , 'age', 'gender'] # names of the metadata columns to be merged

participants_file = r"D:\EEGs\PhD_datasets\BIDS\finland\participants.tsv"
participants_df = pd.read_csv(participants_file, sep="\t", converters = {'subject': str})
participants_df.columns= participants_df.columns.str.lower() # To set all the column names in lower case
participants_df['participant_id'] = participants_df['participant_id'].str.replace('sub-', 'f_')  # Drop the "sub-" prefix used for BIDS


entropies_file = r"D:/Papers/FDA_spectral/review/reanalysis/finland/entropies_fda.csv"
entropies_df = pd.read_csv(entropies_file, sep=",", converters = {'subject': str})
entropies_df.rename(columns = {'subject':'participant_id'}, inplace = True)
df_1 = pd.DataFrame(pd.merge(participants_df[metadata_cols], entropies_df, on="participant_id")) # Merge metadata columns with the features into a new data frame
df_1['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (f_ for Finland)


bandpowers_file = r"D:/Papers/FDA_spectral/review/reanalysis/finland/bandpowers_fda.csv"
bandpowers_df = pd.read_csv(bandpowers_file, sep=",", converters = {'subject': str})
bandpowers_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_2 = pd.DataFrame(pd.merge(df_1, bandpowers_df, on=["participant_id", 'channel', 'center', 'epoch'])) # Merge metadata columns with the features into a new data frame
df_2['participant_id'] = '' + df_1['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for Finland)

df_file = r"D:/Papers/FDA_spectral/review/reanalysis/finland/df_fda.csv"
df_df = pd.read_csv(df_file, sep=",", converters = {'subject': str})
df_df.rename(columns = {'subject':'participant_id'}, inplace = True) 
df_3 = pd.DataFrame(pd.merge(df_2, df_df, on=["participant_id", 'channel', 'center', 'epoch'])) # Merge metadata columns with the features into a new data frame
df_3['participant_id'] = '' + df_2['participant_id'].astype(str) # Add a prefix in the participant_id with the initials of the batch center (fin for Finland)

finland_df = df_3

finland_df.to_csv('D:/Papers/FDA_spectral/review/reanalysis/finland/finland_fda.csv', index=False)

In [250]:
all_centers = pd.concat([medellin_df, california_df, iowa_df, finland_df], axis=0)
all_centers.to_csv('D:/Papers/FDA_spectral/review/reanalysis/allcenters_fda.csv', index=False)

#### ComBat

In [2]:
allcenters_file = r"D:/Papers/FDA_spectral/review/reanalysis/allcenters_averaged.csv"
allcenters_df = pd.read_csv(allcenters_file, sep=",")

In [3]:
# Log 10 transformation of the spectral features
allcenters_df[["delta","theta", "alpha","beta", "pre_alpha","slow_theta","alpha_theta"]] = allcenters_df[["delta","theta", "alpha","beta", "pre_alpha","slow_theta","alpha_theta"]].apply(np.log)

In [4]:
common_ch = ["Fp1", "Fp2", "O1", "O2", "FC2", "Fz", "F4", "P3", "Oz",
"F7", "CP6", "FC1", "P8", "T7", "Cz", "F3",  "CP2", "FC6",
"C3", "FC5", "CP5", "AF4", "F8", "P4",  "CP1", "C4", "AF3",
"T8",  "P7"]

common_ch_df = allcenters_df[allcenters_df['channel'].isin(common_ch)] 


Some minor transformations on tabular data

In [5]:
merged_df = common_ch_df
### Group the channel * bandpower values by participant_id (having each participant in a row and all bandpower*channels in the columns)
df_log_w = pd.pivot(merged_df, index = ['participant_id', 'group' , 'age', 'gender', 'center'], columns = ['channel'],values = ['permutation', 'sample_ent', 'approximate', 'svd_ent', 'higuchi_fd',
       'hjort_mobility', 'hjort_complexity', 'detrended_fluct', 'katz_fd',
       'petrosian_fd', 'delta', 'theta', 'alpha', 'beta', 'pre_alpha',
       'slow_theta', 'alpha_theta', 'df_ep', 'dfv_ave'])
df_log_w.columns = ['_'.join(col).strip() for col in df_log_w.columns.values]
df_log_w = df_log_w.reset_index(drop = False)

In [6]:
original_metadata = df_log_w.iloc[:, 0:5]  ## As all bandpowers are considered as "data" input for ComBat, let's take the remaining metadata from the original file (i.e. participant_id, group, age, gender, center)

In [7]:
### Recode categorical variables into numeric for ComBat
gender=["f", "m"]
dummy_vars=[1,2]
df_log_w['gender'] = df_log_w['gender'].replace(gender, dummy_vars)

center=["medellin", "california", "iowa", "finland" ]
dummy_vars=[1,2,3,4]
df_log_w['center'] = df_log_w['center'].replace(center, dummy_vars)

group=["hc", "pd"]
dummy_vars=[0,1]
df_log_w['group'] = df_log_w['group'].replace(group, dummy_vars)

In [8]:
### Specifying the batch ('center' variable) as well as biological covariates to preserve (i.e. gender, group, age):

covars = {'batch':df_log_w['center'],
          'gender':df_log_w['gender'],
          'group': df_log_w['group'],
          'age': df_log_w['age']}
covars = pd.DataFrame(covars)


# To specify names of the variables that are categorical:
categorical_cols = ['gender', 'group']

# To specify the name of the variable that encodes for the scanner/batch covariate:
batch_col = 'batch'

In [9]:
### Transpose the input data matrix to get features in rows and subjects in columns
data = df_log_w.loc[:, ~df_log_w.columns.isin(['participant_id','center', 'age', 'group', 'gender'])].T.reset_index(drop = True).to_numpy()

In [10]:
from neuroCombat import neuroCombat

In [11]:
### Harmonization step:
data_combat = neuroCombat(dat=data,
    covars=covars,
    batch_col=batch_col,
    categorical_cols=categorical_cols)["data"]

[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data


In [12]:
# Reorganize the data into a new dataframe with metadata
column_names = df_log_w.columns[5:]  # HAY ALGUNA FORMA MÁS ELEGANTE DE HACER ESTO????
combat_df = pd.DataFrame(data_combat.T, columns = column_names + '_combat')
harmonized_df = pd.concat([original_metadata, combat_df], axis=1)

In [13]:
harmonized_df

,participant_id,group,age,gender,center,permutation_AF3_combat,permutation_AF4_combat,permutation_C3_combat,permutation_C4_combat,permutation_CP1_combat,...,dfv_ave_Fz_combat,dfv_ave_O1_combat,dfv_ave_O2_combat,dfv_ave_Oz_combat,dfv_ave_P3_combat,dfv_ave_P4_combat,dfv_ave_P7_combat,dfv_ave_P8_combat,dfv_ave_T7_combat,dfv_ave_T8_combat
0,c_hc1,hc,54,f,california,0.667834,0.658437,0.667888,0.659917,0.651339,...,2.663634,0.588233,1.146457,1.746934,0.490106,1.792724,1.540443,1.722624,2.778022,3.201034
1,c_hc10,hc,59,f,california,0.673258,0.660710,0.617884,0.613931,0.611036,...,2.918153,3.381257,3.281868,3.394562,2.765158,2.558767,2.705988,2.919022,2.371688,3.329466
2,c_hc18,hc,57,m,california,0.640851,0.647824,0.648204,0.632482,0.618726,...,0.723633,0.956768,-0.182314,-0.297822,0.877004,0.330577,0.659857,0.281633,1.071257,0.925180
3,c_hc2,hc,50,f,california,0.675715,0.673883,0.648607,0.663714,0.632920,...,1.566172,3.126102,2.961328,3.301006,2.471145,2.130065,3.047782,2.586142,1.718888,2.066446
4,c_hc20,hc,68,m,california,0.694435,0.694208,0.672710,0.686304,0.674234,...,0.854095,2.566017,1.304543,1.589918,1.418800,1.466741,2.125938,1.569561,2.980835,1.482425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,m_068,pd,60,m,medellin,0.604328,0.617471,0.595472,0.590826,0.580654,...,0.870956,0.641769,0.720923,0.925196,0.622637,0.856346,0.516411,0.618122,0.634471,0.655331
165,m_069,pd,52,m,medellin,0.659462,0.653350,0.637423,0.666435,0.616402,...,0.583527,0.757115,0.642039,0.640764,0.934423,0.750217,0.617790,0.729631,0.753148,0.864995
166,m_070,pd,63,f,medellin,0.620112,0.638948,0.612789,0.613585,0.612564,...,1.736198,1.479712,1.884091,1.888825,1.275036,1.602131,1.359360,1.019058,1.022018,0.474155
167,m_071,pd,62,m,medellin,0.697336,0.689252,0.686567,0.701063,0.667143,...,1.654095,2.058555,1.939543,1.976158,2.505333,2.845986,1.896236,1.806186,2.592260,2.200423


In [14]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from xgboost import XGBClassifier

In [15]:
data = harmonized_df
data_xgb = data.drop(columns=['participant_id', 'age', 'gender', 'center'])

group=["hc", "pd"]
dummy_vars=[0,1]
data_xgb['group'] = data_xgb['group'].replace(group, dummy_vars)

In [19]:
from sklearn.model_selection import train_test_split

X = data_xgb
y = data_xgb['group']
X = X.drop(['group'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [20]:
lr = XGBClassifier(random_state=42)

sfs = SFS(lr, 
          k_features="parsimonious", 
          forward=False, 
          floating=False, 
          scoring='f1',
          verbose=2,
          cv=5,
          n_jobs = -1)

sfs = sfs.fit(X_train, y_train)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   40.7s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 551 out of 551 | elapsed:  2.4min finished

[2023-01-14 13:13:23] Features: 550/1 -- score: 0.6615164835164835[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   38.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 550 out of 550 | elapsed:  2.4min finished

[2023-01-14 13:15:45] Features: 549/1 -- score: 0.6835164835164835[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   38.9s


In [290]:
#SFS "best"
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (40,10)
fig = plot_sfs(sfs.get_metric_dict(), kind='std_err')
plt.title('Sequential Forward Selection (w. StdErr)')
plt.grid()
plt.show()
print(sfs.k_feature_names_)
print(sfs.k_score_)

('df_ep_T7_combat',)
0.8780662525879919


In [18]:
#SFS "parsimonious"
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (40,10)
fig = plot_sfs(sfs.get_metric_dict(), kind='std_err')
plt.title('Sequential Forward Selection (w. StdErr)')
plt.grid()
plt.show()
print(sfs.k_feature_names_)
print(sfs.k_score_)

('df_ep_P8_combat',)
0.8673171390013495


In [300]:
#SBS "best"
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (40,10)
fig = plot_sfs(sfs.get_metric_dict(), kind='std_err')
plt.title('Sequential Forward Selection (w. StdErr)')
plt.grid()
plt.show()
print(sfs.k_feature_names_)
print(sfs.k_score_)

('permutation_C4_combat', 'permutation_Oz_combat', 'permutation_T8_combat', 'sample_ent_Fp1_combat', 'svd_ent_Fp2_combat', 'petrosian_fd_F4_combat', 'delta_C4_combat', 'delta_FC5_combat', 'delta_P8_combat', 'theta_AF4_combat', 'theta_C4_combat', 'theta_Fp2_combat', 'theta_O1_combat', 'theta_P3_combat', 'slow_theta_Fp2_combat', 'df_ep_AF4_combat', 'df_ep_F4_combat', 'df_ep_FC6_combat', 'dfv_ave_P3_combat')
0.8630186335403727


In [21]:
#SBS "parsimonious"
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (40,10)
fig = plot_sfs(sfs.get_metric_dict(), kind='std_err')
plt.title('Sequential Forward Selection (w. StdErr)')
plt.grid()
plt.show()
print(sfs.k_feature_names_)
print(sfs.k_score_)

('hjort_mobility_AF3_combat', 'hjort_mobility_C3_combat', 'hjort_mobility_T7_combat', 'hjort_complexity_FC2_combat', 'petrosian_fd_Oz_combat', 'theta_F8_combat', 'theta_O1_combat', 'theta_T8_combat', 'pre_alpha_Fz_combat', 'pre_alpha_P4_combat', 'pre_alpha_P7_combat', 'slow_theta_CP2_combat', 'slow_theta_Cz_combat', 'slow_theta_Fp2_combat', 'df_ep_CP6_combat')
0.8349811155028547


In [27]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.model_selection import RepeatedKFold
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve


In [22]:
selected_feats = list(sfs.k_feature_names_)

Xsel = X_test[selected_feats]
Xsel

,hjort_mobility_AF3_combat,hjort_mobility_C3_combat,hjort_mobility_T7_combat,hjort_complexity_FC2_combat,petrosian_fd_Oz_combat,theta_F8_combat,theta_O1_combat,theta_T8_combat,pre_alpha_Fz_combat,pre_alpha_P4_combat,pre_alpha_P7_combat,slow_theta_CP2_combat,slow_theta_Cz_combat,slow_theta_Fp2_combat,df_ep_CP6_combat
167,0.370368,0.283799,0.345356,2.140708,1.008117,-2.355057,-1.844949,-1.738352,-1.705751,-2.207026,-1.991441,-2.647632,-2.549242,-2.752998,5.898072
89,0.218256,0.211568,0.214874,1.623126,1.005801,-2.284443,-1.861062,-2.177803,-2.158693,-2.494347,-2.605899,-3.632809,-3.570802,-3.578888,9.647409
107,0.268000,0.287960,0.360448,1.973032,1.008571,-3.985878,-2.805900,-3.020745,-2.295745,-2.277234,-2.332361,-2.982561,-2.916887,-3.061305,7.396211
71,0.310208,0.314930,0.256518,1.836223,1.005778,-1.809233,-1.973618,-2.047666,-2.476827,-2.484309,-2.625720,-3.333898,-3.292723,-3.613580,9.628362
20,0.347033,0.226707,0.264241,1.641184,1.006970,-2.185319,-2.617464,-2.151228,-2.097801,-2.224696,-2.233013,-2.707799,-2.377811,-3.492283,9.202580
108,0.184632,0.196643,0.201898,1.887473,1.004360,-2.215919,-3.033811,-2.555847,-2.600505,-3.308521,-2.991923,-3.207244,-3.347998,-2.754133,9.891278
111,0.207503,0.242497,0.376181,2.118103,1.007464,-2.455029,-2.064860,-2.296979,-2.061499,-2.319043,-2.439207,-3.572784,-3.619172,-3.667695,7.895038
161,0.106116,0.119975,0.118083,3.658324,1.008124,-1.302140,-1.437609,-1.053604,-1.693681,-1.616634,-1.679328,-2.362402,-2.250550,-2.534440,5.898072
99,0.246770,0.330024,0.346363,1.900588,1.007756,-2.334901,-2.307195,-2.187880,-2.868153,-3.324116,-2.877285,-3.273224,-3.461978,-3.187795,9.392451
83,0.298646,0.355951,0.413941,1.425264,1.007792,-3.050985,-1.557720,-2.036849,-3.088374,-2.291043,-1.948077,-3.146002,-3.161133,-3.577326,8.121513


In [28]:
from sklearn.model_selection import StratifiedKFold

cv    = StratifiedKFold(n_splits=5, shuffle = True, random_state = 42)
folds = [(train,test) for train, test in cv.split(Xsel, y_test)]
metrics = ['auc', 'fpr', 'tpr', 'thresholds']
results = {
    'train': {m:[] for m in metrics},
    'val'  : {m:[] for m in metrics},
    'test' : {m:[] for m in metrics}
}

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [30]:
params = {
    'objective'   : 'binary:logistic',
    'eval_metric' : 'logloss'
}

In [31]:
#SBS parsimonious
plt.rcParams["figure.figsize"] = (10,10)
dtest = xgb.DMatrix(Xsel, label=y_test)
for train, test in tqdm(folds, total=len(folds)):
    dtrain = xgb.DMatrix(Xsel.iloc[train,:], label=y_test.iloc[train])
    dval   = xgb.DMatrix(Xsel.iloc[test,:], label=y_test.iloc[test])
    model  = xgb.train(
        dtrain                = dtrain,
        params                = params, 
        evals                 = [(dtrain, 'train'), (dval, 'val')],
        num_boost_round       = 1000,
        verbose_eval          = False,
        early_stopping_rounds = 10,
    )
    sets = [dtrain, dval, dtest]
    for i,ds in enumerate(results.keys()):
        y_preds              = model.predict(sets[i])
        labels               = sets[i].get_label()
        fpr, tpr, thresholds = roc_curve(labels, y_preds)
        results[ds]['fpr'].append(fpr)
        results[ds]['tpr'].append(tpr)
        results[ds]['thresholds'].append(thresholds)
        results[ds]['auc'].append(roc_auc_score(labels, y_preds))
kind = 'test'
c_fill      = 'rgba(52, 152, 219, 0.2)'
c_line      = 'rgba(52, 152, 219, 0.5)'
c_line_main = 'rgba(41, 128, 185, 1.0)'
c_grid      = 'rgba(189, 195, 199, 0.5)'
c_annot     = 'rgba(149, 165, 166, 0.5)'
c_highlight = 'rgba(192, 57, 43, 1.0)'
fpr_mean    = np.linspace(0, 1, 100)
interp_tprs = []
for i in range(4):
    fpr           = results[kind]['fpr'][i]
    tpr           = results[kind]['tpr'][i]
    interp_tpr    = np.interp(fpr_mean, fpr, tpr)
    interp_tpr[0] = 0.0
    interp_tprs.append(interp_tpr)
tpr_mean     = np.mean(interp_tprs, axis=0)
tpr_mean[-1] = 1.0
tpr_std      = np.std(interp_tprs, axis=0)
tpr_upper    = np.clip(tpr_mean+tpr_std, 0, 1)
tpr_lower    = tpr_mean-tpr_std
auc          = np.mean(results[kind]['auc'])
fig = go.Figure([
    go.Scatter(
        x          = fpr_mean,
        y          = tpr_upper,
        line       = dict(color=c_line, width=1),
        hoverinfo  = "skip",
        showlegend = False,
        name       = 'upper'),
    go.Scatter(
        x          = fpr_mean,
        y          = tpr_lower,
        fill       = 'tonexty',
        fillcolor  = c_fill,
        line       = dict(color=c_line, width=1),
        hoverinfo  = "skip",
        showlegend = False,
        name       = 'lower'),
    go.Scatter(
        x          = fpr_mean,
        y          = tpr_mean,
        line       = dict(color=c_line_main, width=2),
        hoverinfo  = "skip",
        showlegend = True,
        name       = f'AUC: {auc:.3f}')
])
fig.add_shape(
    type ='line', 
    line =dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)
fig.update_layout(
    template    = 'plotly_white', 
    title_x     = 0.5,
    xaxis_title = "1 - Specificity",
    yaxis_title = "Sensitivity",
    width       = 600,
    height      = 600,
    legend      = dict(
        yanchor="bottom", 
        xanchor="right", 
        x=0.95,
        y=0.01,
    )
)
fig.update_yaxes(
    range       = [0, 1],
    gridcolor   = c_grid,
    scaleanchor = "x", 
    scaleratio  = 1,
    linecolor   = 'black')
fig.update_xaxes(
    range       = [0, 1],
    gridcolor   = c_grid,
    constrain   = 'domain',
    linecolor   = 'black')

  0%|          | 0/5 [00:00<?, ?it/s]

In [296]:
plt.rcParams["figure.figsize"] = (10,10)
dtest = xgb.DMatrix(Xsel, label=y_test)
for train, test in tqdm(folds, total=len(folds)):
    dtrain = xgb.DMatrix(Xsel.iloc[train,:], label=y_test.iloc[train])
    dval   = xgb.DMatrix(Xsel.iloc[test,:], label=y_test.iloc[test])
    model  = xgb.train(
        dtrain                = dtrain,
        params                = params, 
        evals                 = [(dtrain, 'train'), (dval, 'val')],
        num_boost_round       = 1000,
        verbose_eval          = False,
        early_stopping_rounds = 10,
    )
    sets = [dtrain, dval, dtest]
    for i,ds in enumerate(results.keys()):
        y_preds              = model.predict(sets[i])
        labels               = sets[i].get_label()
        fpr, tpr, thresholds = roc_curve(labels, y_preds)
        results[ds]['fpr'].append(fpr)
        results[ds]['tpr'].append(tpr)
        results[ds]['thresholds'].append(thresholds)
        results[ds]['auc'].append(roc_auc_score(labels, y_preds))
kind = 'test'
c_fill      = 'rgba(52, 152, 219, 0.2)'
c_line      = 'rgba(52, 152, 219, 0.5)'
c_line_main = 'rgba(41, 128, 185, 1.0)'
c_grid      = 'rgba(189, 195, 199, 0.5)'
c_annot     = 'rgba(149, 165, 166, 0.5)'
c_highlight = 'rgba(192, 57, 43, 1.0)'
fpr_mean    = np.linspace(0, 1, 100)
interp_tprs = []
for i in range(4):
    fpr           = results[kind]['fpr'][i]
    tpr           = results[kind]['tpr'][i]
    interp_tpr    = np.interp(fpr_mean, fpr, tpr)
    interp_tpr[0] = 0.0
    interp_tprs.append(interp_tpr)
tpr_mean     = np.mean(interp_tprs, axis=0)
tpr_mean[-1] = 1.0
tpr_std      = np.std(interp_tprs, axis=0)
tpr_upper    = np.clip(tpr_mean+tpr_std, 0, 1)
tpr_lower    = tpr_mean-tpr_std
auc          = np.mean(results[kind]['auc'])
fig = go.Figure([
    go.Scatter(
        x          = fpr_mean,
        y          = tpr_upper,
        line       = dict(color=c_line, width=1),
        hoverinfo  = "skip",
        showlegend = False,
        name       = 'upper'),
    go.Scatter(
        x          = fpr_mean,
        y          = tpr_lower,
        fill       = 'tonexty',
        fillcolor  = c_fill,
        line       = dict(color=c_line, width=1),
        hoverinfo  = "skip",
        showlegend = False,
        name       = 'lower'),
    go.Scatter(
        x          = fpr_mean,
        y          = tpr_mean,
        line       = dict(color=c_line_main, width=2),
        hoverinfo  = "skip",
        showlegend = True,
        name       = f'AUC: {auc:.3f}')
])
fig.add_shape(
    type ='line', 
    line =dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)
fig.update_layout(
    template    = 'plotly_white', 
    title_x     = 0.5,
    xaxis_title = "1 - Specificity",
    yaxis_title = "Sensitivity",
    width       = 600,
    height      = 600,
    legend      = dict(
        yanchor="bottom", 
        xanchor="right", 
        x=0.95,
        y=0.01,
    )
)
fig.update_yaxes(
    range       = [0, 1],
    gridcolor   = c_grid,
    scaleanchor = "x", 
    scaleratio  = 1,
    linecolor   = 'black')
fig.update_xaxes(
    range       = [0, 1],
    gridcolor   = c_grid,
    constrain   = 'domain',
    linecolor   = 'black')

  0%|          | 0/5 [00:00<?, ?it/s]

In [305]:
plt.rcParams["figure.figsize"] = (10,10)
dtest = xgb.DMatrix(Xsel, label=y_test)
for train, test in tqdm(folds, total=len(folds)):
    dtrain = xgb.DMatrix(Xsel.iloc[train,:], label=y_test.iloc[train])
    dval   = xgb.DMatrix(Xsel.iloc[test,:], label=y_test.iloc[test])
    model  = xgb.train(
        dtrain                = dtrain,
        params                = params, 
        evals                 = [(dtrain, 'train'), (dval, 'val')],
        num_boost_round       = 1000,
        verbose_eval          = False,
        early_stopping_rounds = 10,
    )
    sets = [dtrain, dval, dtest]
    for i,ds in enumerate(results.keys()):
        y_preds              = model.predict(sets[i])
        labels               = sets[i].get_label()
        fpr, tpr, thresholds = roc_curve(labels, y_preds)
        results[ds]['fpr'].append(fpr)
        results[ds]['tpr'].append(tpr)
        results[ds]['thresholds'].append(thresholds)
        results[ds]['auc'].append(roc_auc_score(labels, y_preds))
kind = 'test'
c_fill      = 'rgba(52, 152, 219, 0.2)'
c_line      = 'rgba(52, 152, 219, 0.5)'
c_line_main = 'rgba(41, 128, 185, 1.0)'
c_grid      = 'rgba(189, 195, 199, 0.5)'
c_annot     = 'rgba(149, 165, 166, 0.5)'
c_highlight = 'rgba(192, 57, 43, 1.0)'
fpr_mean    = np.linspace(0, 1, 100)
interp_tprs = []
for i in range(4):
    fpr           = results[kind]['fpr'][i]
    tpr           = results[kind]['tpr'][i]
    interp_tpr    = np.interp(fpr_mean, fpr, tpr)
    interp_tpr[0] = 0.0
    interp_tprs.append(interp_tpr)
tpr_mean     = np.mean(interp_tprs, axis=0)
tpr_mean[-1] = 1.0
tpr_std      = np.std(interp_tprs, axis=0)
tpr_upper    = np.clip(tpr_mean+tpr_std, 0, 1)
tpr_lower    = tpr_mean-tpr_std
auc          = np.mean(results[kind]['auc'])
fig = go.Figure([
    go.Scatter(
        x          = fpr_mean,
        y          = tpr_upper,
        line       = dict(color=c_line, width=1),
        hoverinfo  = "skip",
        showlegend = False,
        name       = 'upper'),
    go.Scatter(
        x          = fpr_mean,
        y          = tpr_lower,
        fill       = 'tonexty',
        fillcolor  = c_fill,
        line       = dict(color=c_line, width=1),
        hoverinfo  = "skip",
        showlegend = False,
        name       = 'lower'),
    go.Scatter(
        x          = fpr_mean,
        y          = tpr_mean,
        line       = dict(color=c_line_main, width=2),
        hoverinfo  = "skip",
        showlegend = True,
        name       = f'AUC: {auc:.3f}')
])
fig.add_shape(
    type ='line', 
    line =dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)
fig.update_layout(
    template    = 'plotly_white', 
    title_x     = 0.5,
    xaxis_title = "1 - Specificity",
    yaxis_title = "Sensitivity",
    width       = 600,
    height      = 600,
    legend      = dict(
        yanchor="bottom", 
        xanchor="right", 
        x=0.95,
        y=0.01,
    )
)
fig.update_yaxes(
    range       = [0, 1],
    gridcolor   = c_grid,
    scaleanchor = "x", 
    scaleratio  = 1,
    linecolor   = 'black')
fig.update_xaxes(
    range       = [0, 1],
    gridcolor   = c_grid,
    constrain   = 'domain',
    linecolor   = 'black')

  0%|          | 0/5 [00:00<?, ?it/s]

In [297]:
import shap

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(Xsel)
shap.summary_plot(shap_values, Xsel, plot_type="bar")

In [32]:
#SBS parsimonious
sfs = pd.DataFrame.from_dict(sfs.get_metric_dict()).T

In [298]:
sfs = pd.DataFrame.from_dict(sfs.get_metric_dict()).T

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
1,"(520,)","[0.9600000000000001, 0.8571428571428571, 0.956...",0.878066,"(df_ep_T7_combat,)",0.123954,0.09644,0.04822
2,"(508, 520)","[0.9600000000000001, 0.8695652173913043, 0.75,...",0.851627,"(df_ep_FC5_combat, df_ep_T7_combat)",0.107034,0.083276,0.041638
3,"(339, 508, 520)","[0.9090909090909091, 0.8695652173913043, 0.727...",0.856822,"(theta_O1_combat, df_ep_FC5_combat, df_ep_T7_c...",0.102528,0.07977,0.039885
4,"(96, 339, 508, 520)","[0.9166666666666666, 0.9090909090909091, 0.727...",0.85202,"(svd_ent_F3_combat, theta_O1_combat, df_ep_FC5...",0.091793,0.071418,0.035709
5,"(96, 339, 508, 517, 520)","[0.9565217391304348, 0.8695652173913043, 0.719...",0.83952,"(svd_ent_F3_combat, theta_O1_combat, df_ep_FC5...",0.098509,0.076643,0.038321
...,...,...,...,...,...,...,...
547,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.8799999999999999, 0.7199999999999999, 0.666...",0.713534,"(permutation_AF3_combat, permutation_AF4_comba...",0.116942,0.090985,0.045492
548,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.8799999999999999, 0.7199999999999999, 0.666...",0.713534,"(permutation_AF3_combat, permutation_AF4_comba...",0.116942,0.090985,0.045492
549,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.8333333333333334, 0.7199999999999999, 0.583...",0.687534,"(permutation_AF3_combat, permutation_AF4_comba...",0.114105,0.088778,0.044389
550,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.8799999999999999, 0.7199999999999999, 0.521...",0.6719,"(permutation_AF3_combat, permutation_AF4_comba...",0.167159,0.130056,0.065028


#### Connectivity

In [10]:
import mne_connectivity
from mne_connectivity.viz import plot_sensors_connectivity

##### California #####

In [138]:
california={
'layout':{'extension':'.fif', 'session':['hc', 'on'], 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['FP1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5', 'P7', 'P3', 'PZ', 'PO3', 'O1', 'OZ', 'O2', 'PO4', 'P4', 'P8', 'CP6', 'CP2', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'AF4', 'FP2', 'FZ', 'CZ'],
    'path':'D:/EEGs/PhD_datasets/BIDS/california/'
}

#Path of the BIDS folder
bids_root = 'D:/EEGs/PhD_datasets/BIDS/california/'
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=california #DEFINE DATASET

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]

print(len(eegs), eegs)

31 ['D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc1\\ses-hc\\eeg\\sub-hc1_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc2\\ses-hc\\eeg\\sub-hc2_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc4\\ses-hc\\eeg\\sub-hc4_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc7\\ses-hc\\eeg\\sub-hc7_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc8\\ses-hc\\eeg\\sub-hc8_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc10\\ses-hc\\eeg\\sub-hc10_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc18\\ses-hc\\eeg\\sub-hc18_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\B

In [141]:
%%time
%%capture
matrices = []

for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')
    
    epochs = mne.read_epochs(eeg_file, preload = True)
    all_channs = epochs.info['ch_names']
    keep_chans = DATASET['ch_names']
    drop = set(all_channs) ^ set(keep_chans)
    epochs.drop_channels(drop)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs = epochs.resample(downsample)
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(28)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    epochs.set_montage("standard_1005")
    matrices = []
    bands = [[1, 4], [4, 8], [8, 13], [13, 30]]
    for band in bands:
        fmin, fmax = band
        
        con = mne_connectivity.spectral_connectivity_epochs(epochs.get_data(),
                                                            method='pli', mode='multitaper', 
                                                            sfreq=epochs.info["sfreq"], fmin=fmin, fmax=fmax, 
                                                            faverage=True, mt_adaptive=False, n_jobs=1)

        # plot_sensors_connectivity(epochs.info, con.get_data(output='dense')[:, :, 0])
        
        
        # mne_connectivity.viz.plot_connectivity_circle(con.get_data(output='dense')[:, :, 0], 
        #                                         node_names= epochs.info["ch_names"], vmin=0.2, 
        #                                         vmax=None, colormap='cubehelix',
        #                                         node_colors= ['greenyellow'])
        matrices.append(con.get_data(output='dense'))


    np.save(f"D:/Papers/FDA_spectral/review/reanalysis/california/connectivity/pli_{subject}.npy", matrices)

Wall time: 51.7 s


In [142]:
matrix = np.load("D:/Papers/FDA_spectral/review/reanalysis/california/connectivity/pli_hc4.npy")


In [143]:
matrix.shape


(4, 32, 32, 1)

In [164]:
# Lista de nombres de los sujetos
subjects = layout.get_subjects()
pd_subjects = [k for k in subjects if 'pd' in k]
hc_subjects = [i for i in subjects if 'hc' in i]

# Crear una lista vacía para guardar las matrices de cada sujeto
pd_matrices = []
hc_matrices = []

# Iterar sobre cada sujeto del grupo A
for subj in pd_subjects:
    # Cargar la matriz de ese sujeto
    matrix = np.load("D:/Papers/FDA_spectral/review/reanalysis/california/connectivity/pli_" + subj + ".npy")
    # Añadir la matriz a la lista de matrices del grupo A
    pd_matrices.append(matrix)
    
# Iterar sobre cada sujeto del grupo B
for subj in hc_subjects:
    # Cargar la matriz de ese sujeto
    matrix = np.load("D:/Papers/FDA_spectral/review/reanalysis/california/connectivity/pli_" + subj + ".npy")
    # Añadir la matriz a la lista de matrices del grupo B
    hc_matrices.append(matrix)

# Concatenar las matrices de los sujetos del grupo A y del grupo B en una única matriz 4-dimensional


all_pd = np.concatenate(pd_matrices, axis=-1)
all_hc = np.concatenate(hc_matrices, axis=-1)

np.save(f"D:/Papers/FDA_spectral/review/reanalysis/california/connectivity/pd_pli.npy", all_pd)
np.save(f"D:/Papers/FDA_spectral/review/reanalysis/california/connectivity/hc_pli.npy", all_hc)

In [163]:
all_pd.shape

(4, 32, 32, 15)

In [194]:
X1 = [np.transpose(x, (2, 1, 0)) for x in all_pd]  # transpose for clustering
X2 = [np.transpose(x, (2, 1, 0)) for x in all_hc]  # transpose for clustering

In [195]:
X2[0].shape

(16, 32, 32)

In [200]:
t, cl, p, h0 = permutation_cluster_test([X1[0], X2[0]])


Using a threshold of 4.182964
stat_fun(H1): min=nan max=nan
Running initial clustering …
Found 34 clusters


C:\Users\acer\AppData\Local\Temp/ipykernel_23696/2336531321.py:1: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  t, cl, p, h0 = permutation_cluster_test([X1[0], X2[0]])
C:\Users\acer\AppData\Local\Temp/ipykernel_23696/2336531321.py:1: RuntimeWarning: Provided stat_fun does not treat variables independently. Setting buffer_size to None.
  t, cl, p, h0 = permutation_cluster_test([X1[0], X2[0]])
100%|██████████| Permuting : 1023/1023 [00:00<00:00, 1530.95it/s]


In [205]:
from mne.stats import permutation_cluster_test

# Cargar matrices del grupo X1 y X2
X1 = [np.transpose(x, (2, 1, 0)) for x in all_pd]  # transpose for clustering
X2 = [np.transpose(x, (2, 1, 0)) for x in all_hc]  # transpose for clustering


# Número de permutaciones
n_permutations = 10000
# Significancia
p_threshold = 0.05

# Inicializar lista vacía para guardar los resultados
results = []

# Iterar sobre las bandas de frecuencia
for i in range(4):
    # Seleccionar matrices de la banda actual
    t, cl, p, h0 = permutation_cluster_test([X1[i], X2[i]], threshold=None)
    # Guardar los resultados en la lista
   


Using a threshold of 4.182964
stat_fun(H1): min=nan max=nan
Running initial clustering …
Found 34 clusters


C:\Users\acer\AppData\Local\Temp/ipykernel_23696/1041467559.py:19: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  t, cl, p, h0 = permutation_cluster_test([X1[i], X2[i]])
c:\Users\acer\anaconda3\lib\site-packages\mne\stats\parametric.py:170: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
C:\Users\acer\AppData\Local\Temp/ipykernel_23696/1041467559.py:19: RuntimeWarning: Provided stat_fun does not treat variables independently. Setting buffer_size to None.
  t, cl, p, h0 = permutation_cluster_test([X1[i], X2[i]])
100%|██████████| Permuting : 1023/1023 [00:00<00:00, 1561.16it/s]


Using a threshold of 4.182964
stat_fun(H1): min=nan max=nan
Running initial clustering …
Found 27 clusters


C:\Users\acer\AppData\Local\Temp/ipykernel_23696/1041467559.py:19: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  t, cl, p, h0 = permutation_cluster_test([X1[i], X2[i]])
C:\Users\acer\AppData\Local\Temp/ipykernel_23696/1041467559.py:19: RuntimeWarning: Provided stat_fun does not treat variables independently. Setting buffer_size to None.
  t, cl, p, h0 = permutation_cluster_test([X1[i], X2[i]])
100%|██████████| Permuting : 1023/1023 [00:00<00:00, 1678.69it/s]

Using a threshold of 4.182964
stat_fun(H1): min=nan max=nan
Running initial clustering …
Found 27 clusters



C:\Users\acer\AppData\Local\Temp/ipykernel_23696/1041467559.py:19: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  t, cl, p, h0 = permutation_cluster_test([X1[i], X2[i]])
C:\Users\acer\AppData\Local\Temp/ipykernel_23696/1041467559.py:19: RuntimeWarning: Provided stat_fun does not treat variables independently. Setting buffer_size to None.
  t, cl, p, h0 = permutation_cluster_test([X1[i], X2[i]])
100%|██████████| Permuting : 1023/1023 [00:00<00:00, 1641.08it/s]


Using a threshold of 4.182964
stat_fun(H1): min=nan max=nan
Running initial clustering …
Found 6 clusters


C:\Users\acer\AppData\Local\Temp/ipykernel_23696/1041467559.py:19: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  t, cl, p, h0 = permutation_cluster_test([X1[i], X2[i]])
C:\Users\acer\AppData\Local\Temp/ipykernel_23696/1041467559.py:19: RuntimeWarning: Provided stat_fun does not treat variables independently. Setting buffer_size to None.
  t, cl, p, h0 = permutation_cluster_test([X1[i], X2[i]])
100%|██████████| Permuting : 1023/1023 [00:00<00:00, 1838.24it/s]


In [221]:
from scipy.stats import ttest_ind

# Inicializar lista vacía para guardar los resultados
results = []

# Iterar sobre las bandas de frecuencia
for i in range(4):
    # Seleccionar matrices de la banda actual
    X1_band = X1[i].mean(axis=-1)
    X2_band = X2[i].mean(axis=-1)
    t, p = ttest_ind(X1_band, X2_band, axis=0)
    results.append((t, p))

In [232]:
p_values

[array([0.09848331, 0.00865842, 0.07070292, 0.00890785, 0.02902374,
        0.01140565, 0.48113304, 0.09402169, 0.04800724, 0.15060384,
        0.04289923, 0.67838746, 0.38016958, 0.25655633, 0.0276571 ,
        0.2174548 , 0.26234031, 0.85812923, 0.93181671, 0.23092861,
        0.05030034, 0.51575274, 0.14173964, 0.72190018, 0.40190564,
        0.05964683, 0.63057812, 0.69885645, 0.79728066, 0.09944776,
        0.02105889,        nan]),
 array([0.02531474, 0.02836244, 0.02006561, 0.10301499, 0.06055264,
        0.42770509, 0.44775424, 0.25233622, 0.05961665, 0.93869403,
        0.26614932, 0.02446018, 0.08112308, 0.19888537, 0.19678256,
        0.10645005, 0.17077535, 0.11605241, 0.04469124, 0.15101091,
        0.09347788, 0.4538154 , 0.40559502, 0.14557927, 0.07691042,
        0.06708232, 0.41484545, 0.09579168, 0.3069301 , 0.84565164,
        0.12855322,        nan]),
 array([0.16053927, 0.02230833, 0.14395227, 0.03497015, 0.07018782,
        0.14287938, 0.81260774, 0.20389186, 0.08

In [217]:

# set family-wise p-value
p_accept = 0.05

for i in range(4):
    cluster_stats = permutation_cluster_test([X1[i], X2[i]], n_permutations=1000,
                                                threshold=None)

    T_obs, clusters, p_values, _ = cluster_stats
    good_cluster_inds = np.where(p_values < p_accept)[0]

Using a threshold of 4.182964
stat_fun(H1): min=nan max=nan
Running initial clustering …
Found 34 clusters


C:\Users\acer\AppData\Local\Temp/ipykernel_23696/2166931954.py:5: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  cluster_stats = permutation_cluster_test([X1[i], X2[i]], n_permutations=1000,
C:\Users\acer\AppData\Local\Temp/ipykernel_23696/2166931954.py:5: RuntimeWarning: Provided stat_fun does not treat variables independently. Setting buffer_size to None.
  cluster_stats = permutation_cluster_test([X1[i], X2[i]], n_permutations=1000,
100%|██████████| Permuting : 999/999 [00:00<00:00, 1370.27it/s]
C:\Users\acer\AppData\Local\Temp/ipykernel_23696/2166931954.py:5: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  cluster_stats = permutation_cluster_test([X1[i], X2[i]], n_permutations=1000,


Using a threshold of 4.182964
stat_fun(H1): min=nan max=nan
Running initial clustering …
Found 27 clusters


C:\Users\acer\AppData\Local\Temp/ipykernel_23696/2166931954.py:5: RuntimeWarning: Provided stat_fun does not treat variables independently. Setting buffer_size to None.
  cluster_stats = permutation_cluster_test([X1[i], X2[i]], n_permutations=1000,
100%|██████████| Permuting : 999/999 [00:00<00:00, 1690.59it/s]


Using a threshold of 4.182964
stat_fun(H1): min=nan max=nan
Running initial clustering …
Found 27 clusters


C:\Users\acer\AppData\Local\Temp/ipykernel_23696/2166931954.py:5: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  cluster_stats = permutation_cluster_test([X1[i], X2[i]], n_permutations=1000,
C:\Users\acer\AppData\Local\Temp/ipykernel_23696/2166931954.py:5: RuntimeWarning: Provided stat_fun does not treat variables independently. Setting buffer_size to None.
  cluster_stats = permutation_cluster_test([X1[i], X2[i]], n_permutations=1000,
100%|██████████| Permuting : 999/999 [00:00<00:00, 1769.73it/s]


Using a threshold of 4.182964
stat_fun(H1): min=nan max=nan
Running initial clustering …
Found 6 clusters


C:\Users\acer\AppData\Local\Temp/ipykernel_23696/2166931954.py:5: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  cluster_stats = permutation_cluster_test([X1[i], X2[i]], n_permutations=1000,
C:\Users\acer\AppData\Local\Temp/ipykernel_23696/2166931954.py:5: RuntimeWarning: Provided stat_fun does not treat variables independently. Setting buffer_size to None.
  cluster_stats = permutation_cluster_test([X1[i], X2[i]], n_permutations=1000,
100%|██████████| Permuting : 999/999 [00:00<00:00, 1718.13it/s]


In [218]:
p_values

array([0.997, 0.993, 0.947, 0.904, 0.83 , 1.   ])